In [107]:
import mysql.connector
import mysql_config
from sklearn.neighbors import BallTree
from sklearn.metrics import DistanceMetric
import pandas as pd
from generate_spatial_location import generate_spatial_location
import time
from math import radians
from tqdm import tqdm

### Generate test cases

In [98]:
RADIUS = 1000 * 1000 # 1000km

test_coords = generate_spatial_location(num_samples=10)
test_coords = [{"latitude" : coord.latitude, "longitude" : coord.longitude} for coord in test_coords]

pd.DataFrame(test_coords)

,latitude,longitude
0,-45.615177,128.918880
1,3.2693677,174.406829
2,-3.1414800,41.6408665
3,-39.021237,128.357045
4,-60.770185,-155.182407
5,-2.4184236,107.923392
6,-22.458415,151.211147
7,29.385264,-6.23489428
8,-10.713211,41.9828035
9,16.273357,-130.421662


### Approach 1: Using spatial data type of MySQL

In [99]:
approach_1_result = []

query = """ 

SELECT ST_Distance_Sphere(`position`, st_geomfromtext('point(%s %s)', 4326)) AS `distance_m`, 
       st_longitude(position) as `Longitude`,
       st_latitude(position) as `Latitude`
FROM GeoLocation
HAVING distance_m <= %s; 

"""
try:
    conn = mysql.connector.connect(**mysql_config.config)
    cur = conn.cursor()

    print("Successfully connected to the database")

    for i in tqdm(range(len(test_coords))):
        start_time = time.time()
        cur.execute(query, (test_coords[i]["latitude"], test_coords[i]["longitude"], RADIUS))
        cur.fetchall()
        end_time = time.time()

        approach_1_result.append({
            "Location" : i, 
            "Query time" : end_time - start_time,
            "Row Count" : cur.rowcount
        })

        cur.reset()

    conn.commit()
    print("Query completed!")

    cur.close()
except mysql.connector.Error as Error:
    print("Fail to execute query. Error :\n", Error)
finally:
    if conn:
        conn.close()
    print("Connection is closed")

pd.DataFrame(approach_1_result)

Successfully connected to the database


100%|██████████| 10/10 [01:45<00:00, 10.56s/it]

Query completed!
Connection is closed


,Location,Query time,Row Count
0,0,11.322522,5672
1,1,11.241704,3914
2,2,10.285750,3935
3,3,9.529558,5053
4,4,11.471666,8117
5,5,11.073095,3904
6,6,9.845515,4329
7,7,10.316774,4466
8,8,8.280211,4036
9,9,12.247621,3968


### Approach 2: Use Machine Learning Model

In [118]:
approach_2_result = []

get_all_coords_query = """ 
    SELECT st_latitude(position), st_longitude(position) 
    FROM GeoLocation;
""" 

try:
    conn = mysql.connector.connect(**mysql_config.config)
    cur = conn.cursor()

    # Get all locations
    cur.execute(get_all_coords_query)
    coords = cur.fetchall()
    
    coords = [[radians(coord[0]), radians(coord[1])] for coord in coords]
    
    # Initilize haversine distance. This distance function is used for calculate distance from 2 points on the earth
    dist = DistanceMetric.get_metric("haversine")
    # Create ball tree
    tree = BallTree(coords, metric=dist)

    # Benchmark for approach 2
    for i in tqdm(range(len(test_coords))):
        start_time = time.time()
        # Divide radius by radius of the Earth in meter. Radius of the Earth in meter is 6371000
        count = tree.query_radius([[radians(test_coords[i]["latitude"]), radians(test_coords[i]["longitude"])]], r=RADIUS/6371000, count_only=True)
        end_time = time.time()

        approach_2_result.append({
            "Location" : i, 
            "Query time" : end_time - start_time,
            "Row Count" : count[0]
        })

    conn.commit()

except mysql.connector.Error as Error:
    print(f"Fail to execute database. Error:\n{Error}")
    
finally:
    if conn: 
        conn.close()
    print("Connection is closed")

pd.DataFrame(approach_2_result)

100%|██████████| 10/10 [00:00<00:00, 171.27it/s]

Connection is closed


,Location,Query time,Row Count
0,0,0.012155,5673
1,1,0.005028,3914
2,2,0.004771,3935
3,3,0.005636,5053
4,4,0.005001,8117
5,5,0.005003,3904
6,6,0.005572,4329
7,7,0.004627,4466
8,8,0.004598,4036
9,9,0.005170,3968
